<a href="https://www.kaggle.com/code/marinabalakina/dll30-dz4-3?scriptVersionId=155046268" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# ** Домашнее** задание по теме «Архитектуры свёрточных сетей»

Цель задания: изучить работу с готовыми моделями из torchvision.

Контекст

Вам необходимо подобрать базовую модель для работы по вашей задаче. Вы пробуете обучать различные модели на “ваших” данных. По результатам отберёте лучшую для дальнейшего обучения.

Задание

Вам необходимо провести эксперименты по начальному обучению различных моделей и сравнить результаты.

1.Возьмите датасет EMNIST из torchvision

2. Обучите на нём модели: ResNet 18, VGG 16, Inception v3, DenseNet 161 (с нуля по 10 эпох)

3. Сведите результаты обучения моделей (графики лоса) в таблицу и сравните их.


Задание со звездочкой*

* Выполните то же задание, используя датасет hymenoptera_data

Инструкция к выполнению задания

* Загрузите датасет, посмотрите примеры картинок в нём и проверьте наличествующие классы и их дисбаланс.

* Создайте модель текущего типа, используя интерфейс torchvision для нужного количества классов.

* Обучите модель с нуля 10 эпох. Фиксируйте значение функции потерь в список для последующего отображения.

Повторите пункты 2 и 3 для всех указанных вариантов моделей.

Формат сдачи работы

Прикрепите ссылку на готовое решение в личном кабинете. Работу можно отправлять в виде ссылки на python-ноутбук из GitHub, Google Colaboratory или аналогичных платформ. Не забудьте открыть доступ на просмотр и комментирование.

Критерии оценивания
По итогу выполнения задания вы получите зачёт.

Задание считается выполненным, если:

вы обучили каждую модель до некоторого улучшения качества

составлена таблица обучения для сравнения

Задание будет отправлено на доработку, если:

использованы не все типы моделей

не составлена сводная таблица с результатами

# Импорт библиотек и пользовательские функции

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
import torch
from torch import nn
import torchvision as tv # consists of popular datasets, model architectures, and common image transformations for computer vision - для работы с предобученными нейросетями
import time

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [5]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    train_accuracy, train_losses, test_accuracy =[], [], []
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')
        train_accuracy.append(train_acc_sum / n)
        train_losses.append(train_l_sum / n)
        test_accuracy.append(test_acc)
    return train_accuracy, train_losses, test_accuracy

In [6]:
BATCH_SIZE = 256
# Переводим картинки в 224х224 и в тензор
transoforms = tv.transforms.Compose([

    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.EMNIST('.', split='mnist', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.EMNIST('.', split='mnist', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

# 2. Обучение моделей

## 2.3. Inception v3

In [7]:
class Inception(nn.Module):
    def __init__(self, ic, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        self.p1_1 = nn.Sequential(nn.Conv2d(ic, c1, kernel_size=1), nn.ReLU())
        self.p2_1 = nn.Sequential(nn.Conv2d(ic, c2[0], kernel_size=1), nn.ReLU())
        self.p2_2 = nn.Sequential(nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1), nn.ReLU())
        self.p3_1 = nn.Sequential(nn.Conv2d(ic, c3[0], kernel_size=1), nn.ReLU())
        self.p3_2 = nn.Sequential(nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2), nn.ReLU())
        self.p4_1 = nn.Sequential(nn.MaxPool2d(3, stride=1, padding=1))
        self.p4_2 = nn.Sequential(nn.Conv2d(ic, c4, kernel_size=1), nn.ReLU())

    def forward(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        # Concatenate the outputs on the channel dimension.
        return torch.cat((p1, p2, p3, p4), dim=1)

In [8]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.ReLU(),
       nn.MaxPool2d(3, stride=2, padding=1))

b2 = nn.Sequential(
       nn.Conv2d(64, 64, kernel_size=1),
       nn.Conv2d(64, 192, kernel_size=3, padding=1),
       nn.MaxPool2d(3, stride=2, padding=1))

b3 = nn.Sequential(
       Inception(192, 64, (96, 128), (16, 32), 32),
       Inception(256, 128, (128, 192), (32, 96), 64),
       nn.MaxPool2d(3, stride=2, padding=1))

In [9]:
b4 = nn.Sequential(
       Inception(480, 192, (96, 208), (16, 48), 64),
       Inception(512, 160, (112, 224), (24, 64), 64),
       Inception(512, 128, (128, 256), (24, 64), 64),
       Inception(512, 112, (144, 288), (32, 64), 64),
       Inception(528, 256, (160, 320), (32, 128), 128),
       nn.MaxPool2d(3, stride=2, padding=1))

b5 = nn.Sequential(
       Inception(832, 256, (160, 320), (32, 128), 128),
       Inception(832, 384, (192, 384), (48, 128), 128),
       nn.AvgPool2d(7))

net = nn.Sequential(b1, b2, b3, b4, b5, nn.Flatten(), nn.Linear(1024, 10))

In [10]:
net

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Inception(
      (p1_1): Sequential(
        (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
      )
      (p2_1): Sequential(
        (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
      )
      (p2_2): Sequential(
        (0): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
      (p3_1): Sequential(
        (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
      )
      (p3_2): Sequenti

In [11]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(net.parameters(), lr=lr)

In [12]:
train_accuracy, train_losses, test_accuracy  = train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 1.917. Train acc: 0.121. Train Loss: 2.301
Step 10. time since epoch: 8.255. Train acc: 0.103. Train Loss: 2.305
Step 20. time since epoch: 14.540. Train acc: 0.107. Train Loss: 2.304
Step 30. time since epoch: 20.821. Train acc: 0.106. Train Loss: 2.303
Step 40. time since epoch: 27.101. Train acc: 0.102. Train Loss: 2.303
Step 50. time since epoch: 33.374. Train acc: 0.101. Train Loss: 2.303
Step 60. time since epoch: 39.704. Train acc: 0.100. Train Loss: 2.303
Step 70. time since epoch: 45.968. Train acc: 0.099. Train Loss: 2.303
Step 80. time since epoch: 52.247. Train acc: 0.101. Train Loss: 2.303
Step 90. time since epoch: 58.538. Train acc: 0.101. Train Loss: 2.303
Step 100. time since epoch: 64.808. Train acc: 0.102. Train Loss: 2.303
Step 110. time since epoch: 71.092. Train acc: 0.102. Train Loss: 2.303
Step 120. time since epoch: 77.358. Train acc: 0.105. Train Loss: 2.301
Step 130. time since epoch: 83.627. Train acc: 0.106. Train Loss: 2.306
Step 

In [13]:
df_results= pd.DataFrame(columns = ['model', 'train_accuracy', 'train_loss', 'test_accuracy','epoch'])
for i in range(10):
  df_results.loc[len(df_results.index)] = ['inception_v3',  train_accuracy[i], train_losses[i], test_accuracy[i], i]

In [15]:
df_results

,model,train_accuracy,train_loss,test_accuracy,epoch
0,inception_v3,0.306433,1.850410,0.9157,0
1,inception_v3,0.965133,0.124662,0.9753,1
2,inception_v3,0.982700,0.061123,0.9889,2
3,inception_v3,0.987933,0.043086,0.9907,3
4,inception_v3,0.989667,0.035289,0.9905,4
5,inception_v3,0.990850,0.030984,0.9912,5
6,inception_v3,0.992750,0.025528,0.9925,6
7,inception_v3,0.993017,0.024471,0.9929,7
8,inception_v3,0.993833,0.021324,0.9922,8
9,inception_v3,0.993900,0.019963,0.9938,9


In [14]:
df_results.to_csv('inception_v3.csv', index=False)

In [ ]:
torch.cuda.empty_cache()
# model = tv.models.inception_v3(pretrained=True)

In [ ]:
model

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train_accuracy, train_losses, test_accuracy  = train(model, train_iter, test_iter, trainer, num_epochs)

In [ ]:
for i in range(10):
  df_results.loc[len(df_results.index)] = ['inception_v3',  train_accuracy[i], train_losses[i], test_accuracy[i], i]

## 2.4. DenseNet 161

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = tv.models.densenet161(pretrained=True)

In [ ]:
model

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)
train_accuracy, train_losses, test_accuracy  = train(model, train_iter, test_iter, trainer, num_epochs)

In [ ]:
for i in range(10):
  df_results.loc[len(df_results.index)] = ['densenet161',  train_accuracy[i], train_losses[i], test_accuracy[i], i]